<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/8_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [153]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [154]:
file_path = "/content/drive/MyDrive/"
data_name  = "full_rap_df_cleaned_3_classifiers.xlsx"

In [ ]:
data = pd.read_excel(file_path + data_name)

# Classifier's Statistics

In [ ]:
def plot_statistics(data, col, quest):
    # ספירת המקרים השונים בעמודה
    value_counts = data[col].value_counts()
    print(value_counts)
    # יצירת הגרף
    fig, ax = plt.subplots(figsize=(8, 8))
    wedges, texts, autotexts = ax.pie(
    value_counts,
    labels=quest,
    autopct='%1.1f%%',
    textprops=dict(color="black"),
    startangle=90
)

# התאמת גודל הטקסטים
    plt.setp(autotexts, size=10, weight="bold")
    plt.setp(texts, size=12)
    plt.title(f'Distribution of {col}')

    plt.show()

    # הדפסת הנתונים מתחת לגרף
    print(f"Data distribution for '{col}':")

    total = sum(map(lambda count: count, value_counts))

    print(f"Total of {total} cases where checked.")

In [ ]:
plot_statistics(data,'Others_classifier_treshold_prediction',['No','Classified as Others'])

In [ ]:
plot_statistics(data,'first_classifier_treshold_prediction',['Request Denied','Request Granted'])

In [ ]:
plot_statistics(data,'second_classifier_treshold_prediction',['Apeal itself Denied','Apeal itself Granted'])

# Add public and private sides

In [ ]:
sides_path = "sides_values with lables.xlsx"
sides_df = pd.read_excel(file_path+sides_path)

In [ ]:
sides_df["Public Sides"].unique()[5]

In [ ]:
sides_df["Private Sides"].unique()[5]

In [ ]:
public_side = []
private_side = []

In [ ]:
def debug_message(message, value=None):
    if value is not None:
        print(f"{message}: {value}")
    else:
        print(message)

# פונקציה לניקוי ערכים
def safe_str(value):
    """
    Converts a value to a safe string, handling non-string types gracefully.
    """
    try:
        if pd.isna(value) or value == "" or value == []:
            return ""
        if isinstance(value, list):
            return " ".join([str(v) for v in value if isinstance(v, str)]).strip()
        return str(value).strip()
    except Exception as e:
        debug_message("Error converting value to string", value)
        return ""

# פונקציה לניקוי טקסט
def clean_text(text):
    """
    Cleans the text by removing unwanted characters and fixing formatting issues.
    """
    # Remove leading and trailing whitespace
    text = text.strip()

    # Fix \n or n between Hebrew words
    text = re.sub(r'(?<=[\u0590-\u05FF])\s*[n\\n]+\s*(?=[\u0590-\u05FF])', ' ', text)

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    return text

def publc_side(value, sides_df):
    """
    Determines if the value belongs to a public entity based on predefined lists and patterns.
    """
    try:
        value = safe_str(value)
        value_cleaned = clean_text(value)

        # בדיקת מילים קבועות בתור "ציבורי"
        keywords = sides_df["Public Sides"].dropna().unique().tolist()
        public_terms = [
        "רשות מקרקעי ישראל", "מדינת ", "מועצה ", "שירות ה", "היחידה הארצי", "מדינתישראל", "לאכיפת ", "הרשות ל", "חוף ",
        "המשרד ל", 'יועמ"ש', "המשפטי לממשלה", "מנהל מקרקעי", "פרקליטות", "שירות בתי הסוהר", 'שב"ס', 'מע"מ', "מכס ", 'קמ"ט',
        "ועדת תכנון", "מרכז השלטון", "משטרה", "הסנגוריה הציבורית", "מינהל מקרקעי", "עירית ", "רשויות", "איגוד ערים",
        "עיריית ", "מדינת ישראל", "משרד ה", "מקרקעי ישראל", "שר ה", "שרת ה", "מועצת ", 'רנ"ג', "סרן", "מחוז ","מחש",
        "מ דינת ישראל", "שרות בתי הסוהר", 'סג"ם', "(במיל)", "(מיל)", "עריית ", "המחלקה לחקירת שוטרים", "יושבי ראש",
        "רשות ה", "אלוף ", 'יו"ר', "יושב ראש", "ראש עיריית", "ועדה", "צבא", "מפקד", "פיקוד", "משטרת", "משטרה"
    ]
        # אם אחד מהמונחים מופיע, צד ציבורי
        if any(term in value_cleaned for term in public_terms + keywords):
            public_side.append(value_cleaned)
            return 1

        # במידה ואין התאמה, נבדוק לפי אורכים ודפוסים כלליים
        if len(value_cleaned) > 5 and " " in value_cleaned:
            private_side.append(value_cleaned)
            return 0

        return None
    except Exception as e:
        return None


# פונקציה לשיוך צד המדינה
def assign_state_side(value, sides_df):
    """
    Maps the side to public ('ב'), private ('א'), or undefined ('לא מוגדר').
    """
    try:
        value = safe_str(value)
        result = publc_side(value, sides_df)

        if result == 1:
            return "ב"
        elif result == 0:
            return "א"
        else:
            return "לא מוגדר"
    except Exception as e:
        debug_message("Error assigning state side for value", value)
        return "לא מוגדר"


In [ ]:
check_nulls_b = data[data["צד ב'"].isna()]
check_nulls_b["צד א'"].unique()

In [ ]:
# ניקוי כללי של העמודה "צד ב'"
data["צד ב'"] = data["צד ב'"].astype(str).str.strip()  # הסרת רווחים
data["צד ב'"] = data["צד ב'"].replace(r'\s+', ' ', regex=True)  # הסרת רווחים מיותרים
data["צד ב'"] = data["צד ב'"].replace(r'[^\w\s]', '', regex=True)  # הסרת תווים מיוחדים


In [ ]:
data["צד המדינה"] = data["צד ב'"].apply(lambda x: assign_state_side(x, sides_df))


In [ ]:
unknown_values = data[data["צד המדינה"] == "לא מוגדר"]
unknown_values["צד ב'"].unique()

In [ ]:
unknown_values["צד א'"].unique()

In [ ]:
data["צד המדינה"].value_counts()

In [ ]:
for i, value in zip(data[data["צד המדינה"] == "לא מוגדר"].index,
                    data[data["צד המדינה"] == "לא מוגדר"]["צד א'"]):

    if(assign_state_side(value,sides_df)=="ב"):
      print(value)
      data["צד המדינה"][i] = 'א'
      public_side.append(value)
      print("מתוקן:" ,data["צד המדינה"][i])
    elif(assign_state_side(value,sides_df)=="א"):
      print("הפונציה מחזירה על  זה צד א'")
      print("צד המדינה: ",data["צד המדינה"][i])
      print("א: ",data["צד א'"][i])
      print("ב: ",data["צד ב'"][i])
      private_side.append(value)

      ####### לשאול את איילת \ יהונתן - אם בצד א' יש פלוני וצד ב' לא מוגדר, האם זה נחשב מ דינה בצד ב' בהכרח? היא תמיד באיזשהו צד??
    # data["צד המדינה"] = 'ב'
    # print(data["צד המדינה"][i])

In [ ]:
print(set(public_side))
print(set(private_side))

In [ ]:

unknown_values = data[data["צד המדינה"] == "לא מוגדר"]
unknown_values["צד ב'"].unique()


In [ ]:
unknown_values["צד א'"].unique()

In [ ]:
# data[data["צד המדינה"]=='ב']["צד ב'"].unique()

In [ ]:
plot_statistics(data,'צד המדינה',['א','ב',"None"])

In [ ]:
data.to_excel(file_path + "full_rap_df_cleaned_3_classifiers_with_state's_side.xlsx", index=False)